This procedure was used to generate 20 melspectrogram coefficients from the training data. 

First, we took the low-pass filtered files from the training set (done in the preprocessing stage) and then converted each one into a melspectrogram. From those melspectrograms, we extracted 20 coefficients. These 20 coefficients would later be used in both the Random Forest and Convolutional Neural Network models as the training set. 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !unzip  '/content/drive/Shared drives/301FP/unzipped/five_second_files.zip' -d '/content/drive/Shared drives/301FP/unzipped/'

In [0]:
# !unzip  '/content/drive/Shared drives/301FP/LPF_Train_Files_7kHz.zip' -d '/content/drive/Shared drives/301FP/unzipped/train_newfilter_wav/'

In [0]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import librosa.display
import csv

In [0]:
# pip install ffmpeg-python

In [0]:
# pip install audioread

In [0]:
# import ffmpeg
# import audioread
# from audioread import audio_open

Create and save melspectrograms for CNN

In [0]:
cd /content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files/

/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files


In [0]:
# creates the spectrogram images for the test files
cmap = plt.get_cmap('inferno')
for filename in os.listdir('/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files/'):
        y, sr = librosa.load(filename, mono=True, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max), fmax=7000)
        plt.axis('off');
        plt.savefig('/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_img/' + filename[:-4] + '.png')
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
listOfFiles = os.listdir('/content/drive/Shared drives/301FP/unzipped/train_original/301-project.audio_train/')
print(len(listOfFiles))

2757


In [0]:
cd /content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_test_files/

/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_test_files


In [0]:
# creates the spectrogram images for the test files
cmap = plt.get_cmap('inferno')
for filename in os.listdir('/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_test_files/'):
        y, sr = librosa.load(filename, mono=True, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
        plt.axis('off');
        plt.savefig('/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_test_img/' + filename[:-4] + '.png')
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
listOfFiles = os.listdir('/content/drive/Shared drives/301FP/unzipped/raw_shortened_files_max_energy/raw_train_shortened_energy_img/')
print(len(listOfFiles))

Create Dataframe containing melfeatures for the training data correction model 

In [0]:
# filtered = '/content/drive/Shared drives/301FP/unzipped/LPF_test/LPF_Test_Files/'
# images_filt = '/content/drive/Shared drives/301FP/unzipped/LPF_test/flattened_array' #filtered_images to rewrite multi-arrays rather than flattened arrays
# files = os.listdir(filtered)
# num_files = np.size(files)
# print(num_files)

In [0]:
melfeatures = pd.DataFrame(columns=['filename','chroma_stft','rmse','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate', 'mfcc'])

In [0]:
!sudo apt-get install libav-tools

In [0]:
cd /content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files

/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files


In [0]:
for filename in (os.listdir('/content/drive/Shared drives/301FP/unzipped/five_second_files/five_second_train_files/')):
        y, sr = librosa.load(filename) # , mono=True, duration=15
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        mfcc2 = np.apply_along_axis(lambda x: np.mean(x), axis = 1, arr = mfcc)
        rmse = librosa.feature.rmse(y)
        data = [[filename, np.mean(chroma_stft), np.mean(rmse), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr), mfcc2]]
        melfeatures = melfeatures.append(data)

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequen

In [0]:
melfeatures.head()

In [0]:
len(melfeatures)

In [0]:
melfeatures.to_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_test_raw.csv')

In [0]:
 features = melfeatures.iloc[:, :7]

In [0]:
mfcc = melfeatures[7].apply(pd.Series)

In [0]:
mfcc.head()

In [0]:
total = pd.concat([features, mfcc], axis = 1)

In [0]:
total.head()

In [0]:
columns=['filename','chroma_stft','rmse','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20']


In [0]:
total.columns = columns

In [0]:
total.head()

In [0]:
len(total)

In [0]:
total.to_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_testdata_raw.csv')

In [0]:
ls

In [0]:
cd 

In [0]:
cd /content/drive/Shared drives/301FP/unzipped/LPF_train/filtered_files

In [0]:
melfeaturestrain = pd.DataFrame(columns=['filename','chroma_stft','rmse','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate', 'mfcc'])

In [0]:
for filename in (os.listdir('/content/drive/Shared drives/301FP/unzipped/train_img_original2')):
        y, sr = librosa.load(filename, mono=True, duration=10)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        mfcc2 = np.apply_along_axis(lambda x: np.mean(x), axis = 1, arr = mfcc)
        rmse = librosa.feature.rmse(y)
        data = [[filename, np.mean(chroma_stft), np.mean(rmse), np.mean(spec_cent), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr), mfcc2]]
        melfeaturestrain = melfeaturestrain.append(data)

In [0]:
melfeaturestrain.head()

In [0]:
len(melfeaturestrain)

In [0]:
train_features = melfeaturestrain.iloc[:, :7]

In [0]:
mfcctrain = melfeaturestrain[7].apply(pd.Series)

In [0]:
mfcctrain.head()

In [0]:
totaltrain = pd.concat([train_features, mfcctrain], axis = 1)

In [0]:
totaltrain.head()

In [0]:
len(totaltrain)

In [0]:
columns=['filename','chroma_stft','rmse','spectral_centroid','spectral_bandwidth','rolloff','zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20']


In [0]:
totaltrain.columns = columns

In [0]:
totaltrain.to_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_traindata_raw.csv')

In [0]:
totaltrain.head()

In [0]:
len(totaltrain)